In [119]:
# imports
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import warnings
from sklearn.svm import SVC
warnings.filterwarnings('ignore')



In [120]:
glass_df = pd.read_csv(r'..\Cases\Glass Identification\Glass.csv')
glass_df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,building_windows_float_processed


In [ ]:
X= glass_df.drop(columns=['Type'])
y= glass_df['Type']
scaler=StandardScaler()
le= LabelEncoder()
y= le.fit_transform(y)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)
# X_train_scaled=scaler.fit_transform(X_train)
# X_test_scaled=scaler.transform(X_test)

In [122]:
log_reg_l1=LogisticRegression(penalty='l2')
knn=KNeighborsClassifier()  #Create a pipeline for scaling and knn and then pass that pipeline in the voting algo
dtree=DecisionTreeClassifier()


# scaler=StandardScaler().set_output(transform='pandas')
# pi=Pipeline([('scaler',scaler),('knn',knn)])

In [123]:
from sklearn.ensemble import VotingClassifier
voting=VotingClassifier([('LogisticWithL1',log_reg_l1),('knn',knn),('DecisionTree',dtree)],voting='soft',weights=[0.7,0.4,0.1])
voting.fit(X_train,y_train)
y_pred_prob=voting.predict_proba(X_test)
print(log_loss(y_test,y_pred_prob))
voting

0.8169728145818387


VotingClassifier(estimators=[('LogisticWithL1', LogisticRegression()),
                             ('knn', KNeighborsClassifier()),
                             ('DecisionTree', DecisionTreeClassifier())],
                 voting='soft', weights=[0.7, 0.4, 0.1])

In [124]:
voting.named_estimators_

{'LogisticWithL1': LogisticRegression(),
 'knn': KNeighborsClassifier(),
 'DecisionTree': DecisionTreeClassifier()}

## Individual predictions

In [125]:
y_pred_proba=voting.named_estimators_['LogisticWithL1'].predict_proba(X_test)
log_loss(y_test,y_pred_proba)

1.0017140178195436

In [126]:
y_pred_proba=voting.named_estimators_['knn'].predict_proba(X_test)
log_loss(y_test,y_pred_proba)

2.6611900273935607

In [127]:
y_pred_proba=voting.named_estimators_['DecisionTree'].predict_proba(X_test)
log_loss(y_test,y_pred_proba)

11.64487263340708